## **IMAGE** **INPAINTING**

In [ ]:
import numpy as np
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
root_dir = '/content/gdrive/MyDrive/Dataset/train/images'
root_dirm = '/content/gdrive/MyDrive/Dataset/train/masked'

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir,transform=None):
        self.root_dir= root_dir
        self.transform = transform
        self.image_folder = ImageFolder(root_dir)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, index):
        img_path, _ = self.image_folder.imgs[index]

        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img

In [ ]:
class PairedImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir,root_dirm,transform=None):
        self.root_dir = root_dir
        self.root_dirm = root_dirm
        self.transform = transform
        self.image_folder = ImageFolder(root_dir)
        self.mask_folder = ImageFolder(root_dirm)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, index):
        img_path, _ = self.image_folder.imgs[index]
        mask_path,_= self.mask_folder.imgs[index]

        img = Image.open(img_path).convert('RGB')
        masked_img = Image.open(mask_path).convert('RGB')

        if self.transform:
            img = self.transform(img)
            masked_img = self.transform(masked_img)

        mask = torch.all(masked_img == 0, dim=0)

        # Create a new 3D tensor with three channels
        result_img = torch.zeros_like(masked_img)

        # Set white pixels in all channels to 1
        for channel in range(masked_img.shape[0]):
            result_img[channel, mask] = 1

        return img, masked_img,mask

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create the paired dataset
paired_dataset = PairedImageDataset(root_dir,root_dirm, transform=transform)

# Create a DataLoader for batching
batch_size = 32
traingen = DataLoader(paired_dataset, batch_size=batch_size, shuffle=True)

# Create a validation dataset
root_dirv = '/content/gdrive/MyDrive/Dataset/validation/images'
root_dirvm = '/content/gdrive/MyDrive/Dataset/validation/masked'

paired_valdata = PairedImageDataset(root_dirv,root_dirvm, transform=transform)
testgen = DataLoader(paired_valdata, batch_size=batch_size, shuffle=True)

In [ ]:
for i, batch in enumerate(traingen):
    if i >= 1:  # Display only the first batch
        break

    images, masked_images,mask = batch
    images = images.to(device)
    masked_images = masked_images.to(device)
    mask = mask.to(device)
    # Create a grid of images and their masked counterparts
    combined_images = torch.cat([images, masked_images,mask], dim=3)

    # Display the grid using matplotlib
    plt.figure(figsize=(15, 6))
    plt.axis("off")
    plt.title(f"Batch {i + 1}: Images and Masked Images")
    plt.imshow(np.transpose(vutils.make_grid(combined_images, padding=2, normalize=True).cpu(), (1, 2, 0)))
    plt.show()

In [ ]:
class Generator(nn.Module):
    def __init__(self,img_size,latent_dim,channels):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.linear_layer = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_layers = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.linear_layer(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        image = self.conv_layers(out)
        return image

class Discriminator(nn.Module):
    def __init__(self,channels,img_size):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_layers = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        self.ds_size = img_size // 2 ** 4
        self.adverse_layer = nn.Sequential(nn.Linear(128 * self.ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, image):
        out = self.conv_layers(image)
        out = out.view(out.shape[0], -1)
        validity = self.adverse_layer(out)
        return validity

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
epochs = 30
batch_size=64
latent_dim=100
channels = 3
img_size = 256
prior_weight = 0.003
optim_steps = 1500

In [ ]:
def context_loss(masked_images, generated_images, masks, weighted=True):
    return torch.sum(((masked_images-generated_images)**2)*masks)

In [ ]:
# We will initially train GAN on our masked images
print("Starting training ...")
epoch = 0
dataset = ImageDataset(root_dirm,transform=transform) #generate masked images to train the generator on
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

generator = Generator(img_size,latent_dim,channels).cuda()
discriminator =Discriminator(channels,img_size).cuda()
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters())
optimizer_D = optim.Adam(discriminator.parameters())



while epoch < epochs:
    epoch = epoch+1
    total_G_loss = 0.0
    total_D_loss = 0.0

    for i, masked_images in enumerate(dataloader):
        valid = torch.FloatTensor(masked_images.shape[0], 1).fill_(1.0).cuda()
        fake = torch.FloatTensor(masked_images.shape[0], 1).fill_(0.0).cuda()
        masked_images = masked_images.cuda()

        #  Train Generator
        optimizer_G.zero_grad()
        z = torch.FloatTensor(np.random.normal(0, 1, (masked_images.shape[0], latent_dim))).cuda()
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        total_G_loss += g_loss.cpu().detach().numpy()

        #  Train Discriminator
        optimizer_D.zero_grad()
        discriminator_opinion_real = discriminator(masked_images)
        discriminator_opinion_fake = discriminator(gen_imgs.detach())
        real_loss = adversarial_loss(discriminator_opinion_real, valid)
        fake_loss = adversarial_loss(discriminator_opinion_fake, fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        total_D_loss += d_loss.cpu().detach().numpy()

        print(f'Epoch [{epoch+1}/{epoch}] | D loss: {d_loss:.4f} | G loss: {g_loss:.4f}')


    print(
        "[Epoch {}/{}] \t[D loss: {:.3f}] \t[G loss: {:.3f}]".format(
            epoch,epochs, total_D_loss, total_G_loss)
    )

    torch.save({"epoch": epoch,
                "state_dict_G": generator.state_dict(),
                "state_dict_D": discriminator.state_dict(),
                "optimizer_G": optimizer_G.state_dict(),
                "optimizer_D": optimizer_D.state_dict()
                }, '/content/gdrive/MyDrive/gan.pth')

In [ ]:
# Sample images displayed

latent_dim = 100
from torchvision.utils import make_grid
# Load the model and optimizer state_dict
checkpoint = torch.load('/content/gdrive/MyDrive/gan.pth')
generator = Generator(img_size,latent_dim,channels)
generator.load_state_dict(checkpoint['state_dict_G'])

# Set the generator to evaluation mode
generator.eval()

# Generate new images
num_images_to_generate = 20
z_optimum = torch.FloatTensor(np.random.normal(0, 1, (num_images_to_generate, latent_dim)))
generated_images = generator(z_optimum)

# Convert the generated images to a grid for visualization
generated_images_grid = make_grid(generated_images.cpu().detach(), nrow=5, normalize=True)

# Convert the PyTorch tensor to a NumPy array and transpose the dimensions
image_numpy = generated_images_grid.numpy().transpose(1, 2, 0)

# Display the generated image
plt.imshow(image_numpy)
plt.axis('off')
plt.show()

In [ ]:

# Function to alculate optimum latent dimension - not working

from torchvision.utils import save_image
# Using trained GAN model
for i,(original_images,corrupted_images,masks) in enumerate(dataset):
    corrupted_images, masks= corrupted_images, masks
    z_optimum = nn.Parameter(torch.FloatTensor(np.random.normal(0, 1, (corrupted_images.shape[0],latent_dim,))))
    optimizer_inpaint = optim.Adam([z_optimum])

    print("Starting backprop to input ...")
    for epoch in range(optim_steps):
        optimizer_inpaint.zero_grad()
        generated_images = generator(z_optimum)
        discriminator_opinion = discriminator(generated_images)
        c_loss = context_loss(corrupted_images, generated_images,masks) # calculate loss only for masked region
        prior_loss = torch.sum(-torch.log(discriminator_opinion))
        inpaint_loss = c_loss +prior_weight*prior_loss
        inpaint_loss.backward()
        optimizer_inpaint.step()
        print("[Epoch: {}/{}] \t[Loss: \t[Context: {:.3f}] \t[Prior: {:.3f}] \t[Inpaint: {:.3f}]]  \r".format(1+epoch,optim_steps, c_loss,
                                                                            prior_loss, inpaint_loss),end="")
    print("")

        for idx in range(generated_images.shape[0]):
        # Get the parent path of the original image
          original_image_path = traingen.dataset.img_paths[i * traingen.batch_size + idx]
          parent_path = os.path.dirname(original_image_path)

          # Create a directory for each category if it doesn't exist
          category_dir = os.path.join(os.path.basename(parent_path),'generated')
          os.makedirs(category_dir, exist_ok=True)

          # Save the final_img
          save_image(generated_img, os.path.join(category_dir, f'final_img_{i}_{idx}.png'))

In [ ]:
import numpy as np
import cv2
from torchvision.transforms import ToPILImage
# Functions for blending operations

def mix_pixel(pix_1, pix_2, perc):
    return (perc / 255 * pix_1) + ((255 - perc) / 255 * pix_2)

def blend_images(img_orig, img_for_overlay, img_mask):
    if len(img_mask.shape) != 3:
        img_mask = cv2.cvtColor(img_mask, cv2.COLOR_GRAY2BGR)

    img_res = mix_pixel(img_orig, img_for_overlay, img_mask)
    return img_res.astype(np.uint8)

def blend_images_using_mask(img, img_insert, result_img):
    to_pil = ToPILImage()

    img_pil = to_pil(img)
    img_insert_pil = to_pil(img_insert)
    img_insert_mask_pil = to_pil(result_img)

    # Convert PIL Images to NumPy arrays (OpenCV format)
    img_array = np.array(img_pil)
    img_insert_array = np.array(img_insert_pil)
    img_insert_mask_array = np.array(img_insert_mask_pil)

    # Blend images using the provided functions
    img_blended_array = blend_images(img_array, img_insert_array, img_insert_mask_array)

    # Resize images for display (if needed)
    rf = 0.4
    img_array_resized = cv2.resize(img_array, (int(img_array.shape[1] * rf), int(img_array.shape[0] * rf)), interpolation=cv2.INTER_CUBIC)
    img_insert_array_resized = cv2.resize(img_insert_array, (int(img_insert_array.shape[1] * rf), int(img_insert_array.shape[0] * rf)), interpolation=cv2.INTER_CUBIC)
    img_blended_array_resized = cv2.resize(img_blended_array, (int(img_blended_array.shape[1] * rf), int(img_blended_array.shape[0] * rf)), interpolation=cv2.INTER_CUBIC)

    return img_blended_array_resized

In [ ]:
checkpoint = torch.load('/content/gdrive/MyDrive/gan.pth', map_location=torch.device('cpu'))
generator = Generator(img_size,latent_dim,channels)
generator.load_state_dict(checkpoint['state_dict_G'])

# Set the generator to evaluation mode
generator.eval()
generator = generator.to('cpu')
z_optimum = nn.Parameter(torch.FloatTensor(np.random.normal(0, 1, (1,latent_dim,))))

for i,(original_images,corrupted_images,masks) in enumerate(traingen):
    z_optimum = nn.Parameter(torch.FloatTensor(np.random.normal(0, 1, (corrupted_images.shape[0],latent_dim,))))
    generated_images = generator(z_optimum)
    for idx in range(generated_images.shape[0]):
      final_img = blend_images_using_mask(generated_images[idx],original_images[idx],masks[idx])
    # Get the parent path of the original image
      original_image_path = traingen.dataset.img_paths[i * traingen.batch_size + idx]
      parent_path = os.path.dirname(original_image_path)

      # Create a directory for each category if it doesn't exist
      category_dir = os.path.join(os.path.basename(parent_path),'generated')
      os.makedirs(category_dir, exist_ok=True)

      # Save the final_img
      save_image(final_img, os.path.join(category_dir, f'final_img_{i}_{idx}.png'))



# **CLASSIFICATION**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC


In [ ]:
train_dir = '/content/gdrive/MyDrive/Dataset/train/images'
test_dir = '/content/gdrive/MyDrive/Dataset/validation/images'

In [ ]:
#Normalizing the dataset and pre-processing the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset using ImageFolder and apply the transforms
train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=test_dir, transform=transform)


# Define the data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Get the number of classes
num_classes = len(train_dataset.classes)

# Get the class index
class_index = 0  # Replace with the desired class index

# Get the class name corresponding to the class index
for class_index in range (num_classes):
 class_name = train_dataset.classes[class_index]
 print(class_name)

In [ ]:
#loading pre-trained model and only last two layers weights wil be updated

AlexNet_model = models.alexnet(pretrained=True)
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,num_classes)
AlexNet_model.classifier[4].requires_grad_(True)
AlexNet_model.classifier[6].requires_grad_(True)

AlexNet_model.eval()

In [ ]:

from tqdm.auto import tqdm
import torch

# training
def train(model, trainloader, optimizer, criterion, device):
    model.to(device)
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image, labels = image.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()

        loss.backward()
        optimizer.step()

    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

# validation
def validate(model, testloader, criterion, device):
    model.to(device)
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1

            image, labels = data
            image, labels = image.to(device), labels.to(device)  # move data to GPU
            outputs = model(image)
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()

    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
#from utils import save_model, save_plots
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_model.parameters(num_classes), lr=0.001, momentum=0.9)
epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(AlexNet_model, train_loader,
                                              optimizer, criterion,device)
    valid_epoch_loss, valid_epoch_acc = validate(AlexNet_model, val_loader,
                                                 criterion,device)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)

print('TRAINING COMPLETE')

In [ ]:
import matplotlib.pyplot as plt

def plot_curves(train_loss, valid_loss, train_acc, valid_acc):
    # Plot Loss Curve
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Training Loss', color='blue')
    plt.plot(valid_loss, label='Validation Loss', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plot Accuracy Curve
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='Training Accuracy', color='blue')
    plt.plot(valid_acc, label='Validation Accuracy', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_curves(train_loss, valid_loss, train_acc, valid_acc)


In [ ]:
torch.save(AlexNet_model.state_dict(), '/content/gdrive/MyDrive/model_classification.pth')

In [ ]:
model = models.alexnet(pretrained=False)
model.classifier[4] = nn.Linear(4096,1024)
model.classifier[6] = nn.Linear(1024,num_classes)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/model_classification.pth'))
model.eval()

In [ ]:
# removing last linear layer
model.classifier = nn.Sequential(*[model.classifier[i] for i in range(5)])
model.eval()

In [ ]:
import numpy as np

# Function to extract features from a batch of images
def extract_features(model, dataloader):
    model.eval()
    all_features = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            features = model(inputs)
            all_features.append(features.squeeze().numpy())
            all_labels.append(labels.numpy())

    all_features = np.vstack(all_features)
    all_labels = np.concatenate(all_labels)

    return all_features, all_labels

X_train_features, y_train = extract_features(model, train_loader)
X_test_features, y_test = extract_features(model, val_loader)

print("X_train_features shape:", X_train_features.shape)
print("X_test_features shape:", X_test_features.shape)

In [ ]:

def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
# Train an SVM classifier
svm_classifier = SVC(kernel='linear',C=1.0, random_state=42, verbose=False)
svm_classifier.fit(X_train_features, y_train)

# Make predictions on the training set
y_train_pred = svm_classifier.predict(X_train_features)

# Compute training accuracy
training_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", training_accuracy*100)

y_pred = svm_classifier.predict(X_test_features)
# Compute test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_accuracy*100)

plot_confusion_matrix(y_test,y_pred)
print(classification_report(y_test, y_pred))